# Analysis of the Professional Counter Strike Global Offensive Circuit
### CMSC 320 Final Project
### Richard Chen
___

## Introduction and Motivation
____
![](https://seeklogo.com/images/C/csgo-logo-CAA0A4D48A-seeklogo.com.png)


Counter-Strike Global Offensive (CS:GO) is a competitive PC game developed by Valve Corporation in 2012. It is part of a franchise of games stretching back to the release of the first Counter-Strike in 2000. CS:GO is played in a first person perspective, in which two teams of 5 players compete to see who wins a total of 16 rounds the fastest. For the first 15 rounds of the game, one side will play an offensive side (T-side) that is tasked with planting and bomb and having it detonate. The other side is tasked with defusing the bomb or preventing it from being planted at all (CT-side). Both teams can also win the round by eliminating all enemy players. In case both teams draw at the 30th round on 15-15, 6 more rounds are added as overtime; ends if a team wins 4 out of 6 rounds. At any point, there are 7 active maps in the competitive pool. Overall matches are played in a **best of 3 maps** to see who wins the series overall.

Because of the competitive nature of the game, a professional circuit has thrived over the past 8 years. With record player numbers in the base game (1.1 million concurrent players), and increased investment from traditional North American sports franchises, such as the Dallas Cowboys, CS:GO might be a sneak preview into the **future of entertainment**. Having a predictive model of how a competitive CS:GO team can be successful could be worth **millions** as the industry becomes more mainstream. Success in this model could open more opportunities for the broader sport of esports. 

In this tutorial, I will be analyzing a dataset from Kaggle Datasets. I will be going step-by-step through the data pipeline of professional CS:GO teams to determine winning trends and strategies. My goal is to lay the framework and an example for others to build upon when analyzing a competitive multiplayer game. This tutorial and analysis will be written with Python 3 in mind.

## Dataset

The dataset used for this tutorial analysis was found via Kaggle Datasets and can be found [here.](https://www.kaggle.com/mateusdmachado/csgo-professional-matches) Once you download the files from the link, you should find 4 csv files that stores the data we're interested in. This dataset includes data about competitive CS:GO matches between November 2015 to March 2020. The original source where the data was originally scraped from was www.hltv.org.

 - Results.csv: stores data about map scores and team rankings
 - Picks.csv: stores data about team's map picks and vetos during the prematch selection between opposing teams
 - Economy.csv: stores data about round start equipment values for all rounds played
 - Players.csv: stores individual performances of professional players on each map

## Getting Started

First we'll need to import the following python libraries:
 - [Pandas](https://pandas.pydata.org/)
 - [Beautiful Soup (OPTIONAL)](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
 
The syntax to install the libraries is "pip install < library name >"

In [54]:
# Importing the required libraries
import pandas as pd

## Data Scraping
Because the author of the CS:GO Kaggle dataset already scrapped the data off www.hltv.org, I will not be explicitly showing the steps needed to perform that. However, more information can be found by exploring documentation of **Beautiful Soup**, a Python library used to get data and parse a HTML page. [Link here to explore Beautiful Soup.](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Data Tidying
After the data is scrapped, I use the **Pandas library's read_csv function** to load it into a a data structure called a "DataFrame." This will allow me to easily manipulate rows and columns based on their values. A DataFrame will be the basis of making our data usable for the rest of the pipeline. Essentially one row will represent one matchup between teams.

In [97]:
# loading data from our data into a dataframe
df = pd.read_csv("./csgo-professional-matches/results.csv")
# displaying the first few items of our datafram
df

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,2020-03-18,Recon 5,TeamOne,Dust2,0,16,2,2,0,1,0,15,5151,2340454,62,63,0,2,2
1,2020-03-18,Recon 5,TeamOne,Inferno,13,16,2,2,8,6,5,10,5151,2340454,62,63,0,2,2
2,2020-03-18,New England Whalers,Station7,Inferno,12,16,2,1,9,6,3,10,5243,2340461,140,118,12,16,2
3,2020-03-18,Rugratz,Bad News Bears,Inferno,7,16,2,2,0,8,7,8,5151,2340453,61,38,0,2,2
4,2020-03-18,Rugratz,Bad News Bears,Vertigo,8,16,2,2,4,5,4,11,5151,2340453,61,38,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45768,2015-11-05,G2,E-frag.net,Inferno,13,16,2,1,8,7,5,9,1970,2299059,7,16,1,2,2
45769,2015-11-05,G2,E-frag.net,Dust2,16,13,1,1,10,5,6,8,1970,2299059,7,16,1,2,2
45770,2015-11-04,CLG,Liquid,Inferno,16,12,1,1,7,8,9,4,1934,2299011,10,14,16,12,1
45771,2015-11-03,NiP,Dignitas,Train,16,4,1,2,4,1,12,3,1934,2299001,6,12,16,4,1


At this point, we will use functions within the Pandas library to make our DataFrame more usable. One of which is to eliminate excessive data that we will not use in our analysis moving forward. For example in this CS:GO case, we will eliminate matches that did not feature top 20 teams. We will also drop columns that will not be helpful moving forward.

Functions used here:
 - **df = df.drop('column_name',1)** will be used to delete 'column_name' in a Pandas DataFrame. 1 is the axis number (0 for rows and 1 for columns).
 - **df = df.loc[df['column_name']...]** this will be used to filter the DataFrame based on the value of column_name. Could use any boolean statement of comparison in the "..." part 
 - **df['column_name'] = df.apply(lambda row: .... if ... else ...,axis =1)** this is the syntax used to create a new column based on already existing values of a particular row
 - **df.rename(index = str, columns = {"old_column_name": "new_column_name", ....}, inplace = True)** this can be used to rename columns and make it more presentable for the data pipeline further down

In [98]:
# cleaning up data
df = df.drop('starting_ct',1)
df = df.drop('result_1',1)
df = df.drop('result_2',1)
df = df.drop('map_wins_1',1)
df = df.drop('map_wins_2',1)
df = df.drop('event_id',1)
df = df.loc[df['rank_1'] <= 20]
df = df.loc[df['rank_2'] <= 20]

df['MapWinner'] = df.apply(lambda row: row.team_2 if row.map_winner == 2 else row.team_1, axis = 1)
df['SeriesWinner'] = df.apply(lambda row: row.team_2 if row.match_winner == 2 else row.team_1, axis = 1)
df = df.drop('map_winner',1)
df = df.drop('match_winner',1)
df.rename(columns = {'_map':'Map','team_1':'Team_One','team_2':'Team_Two','rank_1':'Team_Ones_Rank','rank_2': 'Team_Twos_Rank', \
                    'ct_1': 'Team_One_CT_Rounds','t_2': 'Team_Two_T_Rounds', 't_1':'Team_One_T_Rounds', 'ct_2':'Team_Two_CT_Rounds'}, inplace = True)
df

,date,Team_One,Team_Two,Map,Team_One_CT_Rounds,Team_Two_T_Rounds,Team_One_T_Rounds,Team_Two_CT_Rounds,match_id,Team_Ones_Rank,Team_Twos_Rank,MapWinner,SeriesWinner
9,2020-03-17,fnatic,BIG,Mirage,9,6,3,10,2340397,5,18,BIG,BIG
21,2020-03-17,fnatic,BIG,Overpass,4,4,12,3,2340397,5,18,fnatic,BIG
26,2020-03-17,fnatic,BIG,Inferno,5,10,2,6,2340397,5,18,BIG,BIG
45,2020-03-16,Vitality,GODSENT,Vertigo,6,9,10,5,2340063,10,17,Vitality,Vitality
46,2020-03-16,Vitality,GODSENT,Nuke,6,3,10,5,2340063,10,17,Vitality,Vitality
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45768,2015-11-05,G2,E-frag.net,Inferno,8,7,5,9,2299059,7,16,E-frag.net,E-frag.net
45769,2015-11-05,G2,E-frag.net,Dust2,10,5,6,8,2299059,7,16,G2,E-frag.net
45770,2015-11-04,CLG,Liquid,Inferno,7,8,9,4,2299011,10,14,CLG,CLG
45771,2015-11-03,NiP,Dignitas,Train,4,1,12,3,2299001,6,12,NiP,NiP


## Sources

[Kaggle CS:GO Professional Matches](https://www.kaggle.com/mateusdmachado/csgo-professional-matches)
[Pandas Python Library](https://pandas.pydata.org/)
[Beautiful Soup]([https://www.crummy.com/software/BeautifulSoup/bs4/doc/](https://www.crummy.com/software/BeautifulSoup/bs4/doc/))
